This notebook attempts to use the [GraphCDR architecture](https://academic.oup.com/bib/article-abstract/23/1/bbab457/6415314?redirectedFrom=fulltext) to predict the drug response of cell lines.

The source code is obtained from https://github.com/BioMedicalBigDataMiningLab/GraphCDR.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# %cd /content/drive/MyDrive/CS6208\ Project/benchmarks/
# !git clone https://github.com/BioMedicalBigDataMiningLab/GraphCDR.git

In [ ]:
# Installing PyG based on https://gist.github.com/ameya98/b193856171d11d37ada46458f60e73e7
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/CS6208 Project/benchmarks/GraphCDR/prog/")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
! pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
torch.manual_seed(0)

In [ ]:
from rdkit import Chem
import deepchem as dc
import time

In [ ]:
benchmark_dir = "/content/drive/MyDrive/CS6208 Project/benchmarks/GraphCDR/"

In [ ]:
pd.read_csv("/content/drive/MyDrive/CS6208 Project/node_features/drug_node_features.csv")

,Unnamed: 0,ChEMBL ID,Name,Synonyms,Type,Max Phase,Molecular Weight,Targets,Bioactivities,AlogP,...,Heavy Atoms,HBA (Lipinski),HBD (Lipinski),#RO5 Violations (Lipinski),Molecular Weight (Monoisotopic),Np Likeness Score,Molecular Species,Molecular Formula,Smiles,Inchi Key
0,15,CHEMBL1237123,DOXACURIUM,DOXACURIUM CATION|DOXACURIUM ION,Small molecule,4.0,1035.24,3.0,19.0,None,...,None,None,None,None,1034.5340,None,None,C56H78N2O16+2,COc1cc(CC2c3c(cc(OC)c(OC)c3OC)CC[N+]2(C)CCCOC(...,GBLRQXKSCRCLBZ-UHFFFAOYSA-N
1,17,CHEMBL1446650,MEBEVERINE HYDROCHLORIDE,COLOFAC|COLOFAC 100|COLOFAC IBS|COLOFAC MR|CSA...,Small molecule,4.0,466.02,15.0,44.0,4.6,...,31,6,0,0,429.2515,-0.6,BASE,C25H36ClNO5,CCN(CCCCOC(=O)c1ccc(OC)c(OC)c1)C(C)Cc1ccc(OC)c...,PLGQWYOULXPJRE-UHFFFAOYSA-N
2,26,CHEMBL1590946,CEFSULODIN SODIUM,ABBOTT-46811|CEFSULODIN SODIUM|CEFSULODIN SODI...,Small molecule,4.0,554.54,21.0,30.0,-1.89,...,36,12,4,2,532.0723,-0.12,ACID,C22H19N4NaO8S2,NC(=O)c1cc[n+](CC2=C(C(=O)[O-])N3C(=O)[C@@H](N...,REACMANCWHKJSM-DWBVFMGKSA-M
3,27,CHEMBL1295,BUTOCONAZOLE,BUTOCONAZOLE|Butoconazole|GYNOFORT|Gynazole-1|...,Small molecule,4.0,411.79,8.0,44.0,6.64,...,25,2,0,1,410.0178,-1.15,NEUTRAL,C19H17Cl3N2S,Clc1ccc(CCC(Cn2ccnc2)Sc2c(Cl)cccc2Cl)cc1,SWLMUYACZKCSHZ-UHFFFAOYSA-N
4,34,CHEMBL1200623,ETHYLESTRENOL,DURABOLIN-O|DURABORAL|ETHYLESTRENOL|ETHYLNANDR...,Small molecule,4.0,288.48,121.0,467.0,5.09,...,21,1,1,1,288.2453,2.11,NEUTRAL,C20H32O,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...,AOXRBFRFYPMWLR-XGXHKTLJSA-N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,4189,CHEMBL1200787,PHENOXYBENZAMINE HYDROCHLORIDE,BENSYLYTE HYDROCHLORIDE|DIBENYLINE|DIBENZYLIN|...,Small molecule,4.0,340.29,78.0,177.0,4.19,...,21,2,0,0,303.1390,-0.67,NEUTRAL,C18H23Cl2NO,CC(COc1ccccc1)N(CCCl)Cc1ccccc1.Cl,VBCPVIWPDJVHAN-UHFFFAOYSA-N
1885,4190,CHEMBL817,TOLAZAMIDE,NSC-70762|TOLAZAMIDE|Tolinase|U-17835,Small molecule,4.0,311.41,159.0,696.0,1.77,...,21,6,2,0,311.1304,-1.26,ACID,C14H21N3O3S,Cc1ccc(S(=O)(=O)NC(=O)NN2CCCCCC2)cc1,OUDSBRTVNLOZBN-UHFFFAOYSA-N
1886,4191,CHEMBL953,ENTACAPONE,COMTESS|Comtan|ENTACAPONE|ENTACAPONE ORION|ENT...,Small molecule,4.0,305.29,49.0,221.0,1.78,...,22,8,2,0,305.1012,-0.89,ACID,C14H15N3O5,CCN(CC)C(=O)/C(C#N)=C/c1cc(O)c(O)c([N+](=O)[O-...,JRURYQJSLYLRLN-BJMVGYQFSA-N
1887,4192,CHEMBL328560,SULTHIAME,Conadil|OSPOLOT|RIKER 594|RIKER-594|SULTHIAME|...,Small molecule,4.0,290.37,27.0,143.0,0.26,...,18,6,2,0,290.0395,-2.28,NEUTRAL,C10H14N2O4S2,NS(=O)(=O)c1ccc(N2CCCCS2(=O)=O)cc1,HMHVCUVYZFYAJI-UHFFFAOYSA-N


In [ ]:
drug=pd.read_csv("/content/drive/MyDrive/CS6208 Project/node_features/drug_node_features.csv", usecols=["Name", "Smiles"])
drug_feature = {}
featurizer = dc.feat.ConvMolFeaturizer()
for tup in zip(drug['Name'], drug['Smiles']):
    # print(tup[1])
    mol=Chem.MolFromSmiles(tup[1])
    X = featurizer.featurize(mol)
    drug_feature[str(tup[0])]=[X[0].get_atom_features(),X[0].get_adjacency_list(),1]

In [ ]:
len(drug_feature)

1889

In [ ]:
cl_mut_df = pd.read_csv("/content/drive/MyDrive/CS6208 Project/link_features/cl_gene_mutations.csv", index_col = 0)
cl_mut_df

,DepMap_ID,HugoSymbol
0,ACH-000839,C1QTNF12
1,ACH-000839,CASZ1
3,ACH-000839,PRAMEF13
4,ACH-000839,PRAMEF18
8,ACH-000839,NBPF1
...,...,...
1387890,ACH-000840,F9
1387891,ACH-000840,MAGEC3
1387892,ACH-000840,SPANXN3
1387893,ACH-000840,MT-ND5


In [ ]:
cl_mut_df["value"] = 1
cl_mut_df

,DepMap_ID,HugoSymbol,value
0,ACH-000839,C1QTNF12,1
1,ACH-000839,CASZ1,1
3,ACH-000839,PRAMEF13,1
4,ACH-000839,PRAMEF18,1
8,ACH-000839,NBPF1,1
...,...,...,...
1387890,ACH-000840,F9,1
1387891,ACH-000840,MAGEC3,1
1387892,ACH-000840,SPANXN3,1
1387893,ACH-000840,MT-ND5,1


In [ ]:
mutation_feature = pd.pivot_table(cl_mut_df, index ="DepMap_ID", columns = "HugoSymbol", values="value").fillna(0)

In [ ]:
response_df = pd.read_csv("/content/drive/MyDrive/CS6208 Project/link_features/cl_drug_response_train.csv", index_col =0)
response_df

,DRUG_NAME,AUC,DepMap_ID
692,NELARABINE,0.986169,ACH-002106
43477,TALAZOPARIB,0.949593,ACH-000367
158415,RUCAPARIB,0.952331,ACH-002148
156506,TRAMETINIB,0.941684,ACH-002345
61734,MITOXANTRONE,0.909827,ACH-000968
...,...,...,...
60768,OLAPARIB,0.955723,ACH-002220
239722,TENIPOSIDE,0.922151,ACH-000999
205355,DABRAFENIB,0.377592,ACH-000814
4746,TAMOXIFEN,0.928190,ACH-000738


In [ ]:
datar=response_df
data_idx = []
for tup in zip(datar['DepMap_ID'],datar['DRUG_NAME'],datar['AUC']):
    data_idx.append((tup[0],str(tup[1]),tup[2]))

In [ ]:
datar_test=pd.read_csv("/content/drive/MyDrive/CS6208 Project/link_features/cl_drug_response_test.csv", index_col =0)
data_idx_test = []
for tup in zip(datar_test['DepMap_ID'],datar_test['DRUG_NAME'],datar_test['AUC']):
    data_idx_test.append((tup[0],str(tup[1]),tup[2]))

In [ ]:
len(data_idx)

33812

In [ ]:
len(data_idx_test)

11271

In [ ]:
#----eliminate ambiguity responses
data_sort=sorted(data_idx, key=(lambda x: [x[0], x[1], x[2]]), reverse=True)
data_tmp=[];data_new=[]
data_idx1 = [[i[0],i[1]] for i in data_sort]
for i,k in zip(data_idx1,data_sort):
    if i not in data_tmp:
        data_tmp.append(i)
        data_new.append(k)
nb_celllines = len(set([item[0] for item in data_new]))
nb_drugs = len(set([item[1] for item in data_new]))
print('All %d pairs across %d cell lines and %d drugs.'%(len(data_new),nb_celllines,nb_drugs))

All 32649 pairs across 966 cell lines and 48 drugs.


In [ ]:
#----eliminate ambiguity responses
data_sort=sorted(data_idx_test, key=(lambda x: [x[0], x[1], x[2]]), reverse=True)
data_tmp=[];data_new_test=[]
data_idx1 = [[i[0],i[1]] for i in data_sort]
for i,k in zip(data_idx1,data_sort):
    if i not in data_tmp:
        data_tmp.append(i)
        data_new_test.append(k)
nb_celllines_test = len(set([item[0] for item in data_new_test]))
nb_drugs_test = len(set([item[1] for item in data_new_test]))
print('All %d pairs across %d cell lines and %d drugs.'%(len(data_new_test),nb_celllines_test,nb_drugs_test))

All 11124 pairs across 965 cell lines and 48 drugs.


In [ ]:
data_combined = data_new + data_new_test
len(data_combined)

43773

In [ ]:
len(data_new), len(data_new_test)

(32649, 11124)

In [ ]:
from data_process import FeatureExtract, cmask, CalculateGraphFeat

In [ ]:
import itertools

In [ ]:
import torch.utils.data as Data
from graphset import *
from scipy.sparse import coo_matrix

In [ ]:
# Update process() to use only mutations
def process_updated(drug_feature, mutation_feature, data_new, data_new_test, data_combined, nb_celllines, nb_drugs):
    #-----construct cell line-drug response pairs
    cellineid = list(set([item[0] for item in data_combined]));cellineid.sort()
    pubmedid = list(set([item[1] for item in data_combined]));pubmedid.sort()
    cellmap = list(zip(cellineid,list(range(len(cellineid)))))
    pubmedmap = list(zip(pubmedid,list(range(len(cellineid),len(cellineid)+len(pubmedid)))))
    cellline_num = np.squeeze([[j[1] for j in cellmap if i[0]==j[0]] for i in data_combined])
    pubmed_num = np.squeeze([[j[1] for j in pubmedmap if i[1]==j[0]] for i in data_combined])
    IC_num = np.squeeze([i[2] for i in data_combined])
    allpairs = np.vstack((cellline_num,pubmed_num,IC_num)).T
    allpairs = allpairs[allpairs[:,2].argsort()]

    #----drug_feature_input
    pubid=[item[0] for item in pubmedmap]
    drug_feature_df=pd.DataFrame(drug_feature).T
    drug_features=drug_feature_df.loc[pubid]
    atom_shape=drug_features[0][0].shape[-1]
    drug_data = FeatureExtract(drug_features)
    #----cell line_feature_input
    cellid=[item[0] for item in cellmap]
    # gexpr_feature=gexpr_feature.loc[cellid]
    cl_with_no_mutation = list(set(cellid) - set(mutation_feature.index))
    cl_with_no_mutation_df = pd.DataFrame(columns = mutation_feature.columns)
    cl_with_no_mutation_df["DepMap_ID"] = cl_with_no_mutation
    cl_with_no_mutation_df.set_index("DepMap_ID", drop=True, inplace=True)
    mutation_feature = pd.concat([mutation_feature, cl_with_no_mutation_df.fillna(0)], axis = 0)
   
    # methylation_feature=methylation_feature.loc[cellid]
    mutation = torch.from_numpy(np.array(mutation_feature,dtype='float32'))
    mutation = torch.unsqueeze(mutation, dim=1)
    mutation = torch.unsqueeze(mutation, dim=1)
    # gexpr = torch.from_numpy(np.array(gexpr_feature,dtype='float32'))
    # methylation = torch.from_numpy(np.array(methylation_feature,dtype='float32'))

    #---compile training set and test set
    drug_set = Data.DataLoader(dataset=GraphDataset(graphs_dict=drug_data),collate_fn=collate,batch_size=nb_drugs,shuffle=False)
    cellline_set = Data.DataLoader(dataset=Data.TensorDataset(mutation),batch_size=nb_celllines,shuffle=False)
    use_independent_testset=True
    if(use_independent_testset == True):
        edge_mask = np.ones(len(data_combined), dtype=bool)
        edge_mask[len(data_new):] = False
        train = allpairs[edge_mask][:, 0:3]
        test = allpairs[~edge_mask][:, 0:3]
        print(train.shape, test.shape)
    else:
        CV_edgemask = cmask(len(allpairs), 0.1, 666)
        cross_validation = allpairs[CV_edgemask][:, 0:3]
        vali_mask = cmask(len(cross_validation), 0.2, 66)
        train = cross_validation[vali_mask][:, 0:3]
        test = cross_validation[~vali_mask][:, 0:3]
    train[:, 1] -= nb_celllines
    test[:, 1] -= nb_celllines
    train_mask = coo_matrix((np.ones(train.shape[0], dtype=bool), (train[:, 0], train[:, 1])),
                            shape=(nb_celllines, nb_drugs)).toarray()
    test_mask = coo_matrix((np.ones(test.shape[0], dtype=bool), (test[:, 0], test[:, 1])),
                           shape=(nb_celllines, nb_drugs)).toarray()
    train_mask = torch.from_numpy(train_mask).view(-1)
    test_mask = torch.from_numpy(test_mask).view(-1)
    if (use_independent_testset == True):
        pos_edge = allpairs 
        complete_pairs = list(itertools.product(list(set(cellline_num)), list(set(pubmed_num))))
        pos_pairs = []
        for i in allpairs:
          pos_pairs.append((i[0], i[1] - nb_celllines))
        neg_pairs = list(set(complete_pairs) - set(pos_pairs))
        neg_edge = []
        for i in neg_pairs:
          neg_edge.append((i[0], i[1], 0))
        neg_edge = np.array(neg_edge)
    else:
        pos_edge = cross_validation[cross_validation[:, 2] == 1, 0:2]
        neg_edge = cross_validation[cross_validation[:, 2] == -1, 0:2]
    pos_edge[:, 1] -= nb_celllines
    # neg_edge[:, 1] -= nb_celllines
    label_pos = coo_matrix((np.ones(pos_edge.shape[0]), (pos_edge[:, 0], pos_edge[:, 1])),
                           shape=(nb_celllines, nb_drugs)).toarray()
    label_pos = torch.from_numpy(label_pos).type(torch.FloatTensor).view(-1)
    if (use_independent_testset == True):
        train_edge = allpairs[edge_mask]
    else:
        train_edge = cross_validation[vali_mask]
    train_edge = np.vstack((train_edge, train_edge[:, [1, 0, 2]]))

    return drug_set,cellline_set,train_edge,label_pos,train_mask,test_mask,atom_shape

In [ ]:
drug_set,cellline_set,train_edge,label_pos,train_mask,test_mask,atom_shape = process_updated(drug_feature, mutation_feature, data_new, data_new_test, data_combined, nb_celllines, nb_drugs)

(32649, 3) (11124, 3)


In [ ]:
train_edge

array([[2.80000e+01, 1.20000e+01, 7.44900e-03],
       [1.12000e+02, 2.00000e+00, 9.72600e-03],
       [7.15000e+02, 4.10000e+01, 9.87100e-03],
       ...,
       [9.00000e+00, 6.03000e+02, 9.72973e-01],
       [1.40000e+01, 9.30000e+01, 9.72977e-01],
       [1.70000e+01, 5.16000e+02, 9.72980e-01]])

In [ ]:
from model import GraphCDR, NodeRepresentation, Summary, Encoder

In [ ]:
from functools import cached_property

In [ ]:
from base_model.GCNConv import GCNConv
from base_model.SGConv import SGConv

In [ ]:
import torch.nn.functional as F

In [ ]:
from torch_geometric.nn import global_max_pool as gmp, global_mean_pool

In [ ]:
import torch.nn as nn

In [ ]:
class NodeRepresentationUpdated(nn.Module):
    def __init__(self, gcn_layer, output, units_list=[256, 256, 256], use_relu=True, use_bn=True,
                 use_GMP=False, use_mutation=True, use_gexpr=False, use_methylation=False):
        super(NodeRepresentationUpdated, self).__init__()
        torch.manual_seed(0)
        # -------drug layers
        self.use_relu = use_relu
        self.use_bn = use_bn
        self.units_list = units_list
        self.use_GMP = use_GMP
        self.use_mutation = use_mutation
        self.use_gexpr = use_gexpr
        self.use_methylation = use_methylation
        self.conv1 = SGConv(gcn_layer, units_list[0])
        self.batch_conv1 = nn.BatchNorm1d(units_list[0])
        self.graph_conv = []
        self.graph_bn = []
        for i in range(len(units_list) - 1):
            self.graph_conv.append(SGConv(units_list[i], units_list[i + 1]))
            self.graph_bn.append(nn.BatchNorm1d((units_list[i + 1])))
        self.conv_end = SGConv(units_list[-1], output)
        self.batch_end = nn.BatchNorm1d(output)
        # --------cell line layers (three omics)
        # -------gexp_layer
        if self.use_gexpr:
          self.fc_gexp1 = nn.Linear(dim_gexp, 256)
          self.batch_gexp1 = nn.BatchNorm1d(256)
          self.fc_gexp2 = nn.Linear(256, output)
        # -------methy_layer
        if self.use_methylation:
          self.fc_methy1 = nn.Linear(dim_methy, 256)
          self.batch_methy1 = nn.BatchNorm1d(256)
          self.fc_methy2 = nn.Linear(256, output)
        # -------mut_layer
        self.cov1 = nn.Conv2d(1, 50, (1, 700), stride=(1, 5))
        self.cov2 = nn.Conv2d(50, 30, (1, 5), stride=(1, 2))
        self.fla_mut = nn.Flatten()
        self.fc_mut = nn.Linear(1470, output)
        # ------Concatenate_three omics
        self.fcat = nn.Linear(100, output)
        self.batchc = nn.BatchNorm1d(100)
        self.reset_para()

    def reset_para(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        return

    def forward(self, drug_feature, drug_adj, ibatch, mutation_data, gexpr_data, methylation_data):
        # -----drug representation
        x_drug = self.conv1(drug_feature, drug_adj)
        x_drug = F.relu(x_drug)
        x_drug = self.batch_conv1(x_drug)
        for i in range(len(self.units_list) - 1):
            x_drug = self.graph_conv[i](x_drug, drug_adj)
            x_drug = F.relu(x_drug)
            x_drug = self.graph_bn[i](x_drug)
        x_drug = self.conv_end(x_drug, drug_adj)
        x_drug = F.relu(x_drug)
        x_drug = self.batch_end(x_drug)
        if self.use_GMP:
            x_drug = gmp(x_drug, ibatch)
        else:
            x_drug = global_mean_pool(x_drug, ibatch)
            
        # -----cell line representation
        # -----mutation representation
        if self.use_mutation:
            x_mutation = torch.tanh(self.cov1(mutation_data))
            x_mutation = F.max_pool2d(x_mutation, (1, 5))
            x_mutation = F.relu(self.cov2(x_mutation))
            x_mutation = F.max_pool2d(x_mutation, (1, 10))
            x_mutation = self.fla_mut(x_mutation)
            x_mutation = F.relu(self.fc_mut(x_mutation))
            # x_mutation = torch.dropout(x_mutation, 0.1, train=False)

        # ----gexpr representation
        if self.use_gexpr:
            x_gexpr = torch.tanh(self.fc_gexp1(gexpr_data))
            x_gexpr = self.batch_gexp1(x_gexpr)
            # x_gexpr = torch.dropout(x_gexpr,0.1, train=False)
            x_gexpr = F.relu(self.fc_gexp2(x_gexpr))

        # ----methylation representation
        if self.use_methylation:
            x_methylation = torch.tanh(self.fc_methy1(methylation_data))
            x_methylation = self.batch_methy1(x_methylation)
            # x_methylation = torch.dropout(x_methylation, 0.1, train=False)
            x_methylation = F.relu(self.fc_methy2(x_methylation))

        # ------Concatenate representations of three omics
        x_cell = F.relu(x_mutation)
        # x_cell = F.relu(self.fcat(x_cell))
        
        #combine representations of cell line and drug
        x_all = torch.cat((x_cell, x_drug), 0)
        x_all = self.batchc(x_all)
        return x_all

In [ ]:
from torch.nn import Parameter

In [ ]:
from my_utiils import *

In [ ]:
EPS = 1e-15
class GraphCDRUpdated(nn.Module):
    def __init__(self, hidden_channels, encoder, summary, feat, index):
        super(GraphCDRUpdated, self).__init__()
        self.hidden_channels = hidden_channels
        self.encoder = encoder
        self.summary = summary
        self.feat = feat
        self.index = index
        self.weight = Parameter(torch.Tensor(hidden_channels, hidden_channels))
        self.act = nn.Sigmoid()
        self.fc = nn.Linear(100, 10)
        self.fd = nn.Linear(100, 10)
        self.reset_parameters()

    def reset_parameters(self):
        reset(self.encoder)
        reset(self.summary)
        glorot(self.weight)
        for m in self.modules():
            if isinstance(m, (nn.Linear)):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, drug_feature, drug_adj, ibatch, mutation_data, gexpr_data, methylation_data, edge):
        #---CDR_graph_edge and corrupted CDR_graph_edge
        pos_edge = torch.from_numpy(edge[edge[:, 2] != 0, 0:2].T).to(torch.long)
        neg_edge = torch.from_numpy(edge[edge[:, 2] == 0, 0:2].T).to(torch.long)
        #---cell+drug node attributes
        feature = self.feat(drug_feature, drug_adj, ibatch, mutation_data, gexpr_data, methylation_data)
        #---cell+drug embedding from the CDR graph and the corrupted CDR graph
        pos_z = self.encoder(feature, pos_edge)
        neg_z = self.encoder(feature, neg_edge)
        #---graph-level embedding (summary)
        summary_pos = self.summary(feature, pos_z)
        summary_neg = self.summary(feature, neg_z)
        #---embedding at layer l
        cellpos = pos_z[:self.index, ]; drugpos = pos_z[self.index:, ]
        #---embedding at layer 0
        cellfea = self.fc(feature[:self.index, ]); drugfea = self.fd(feature[self.index:, ])
        cellfea = torch.sigmoid(cellfea); drugfea = torch.sigmoid(drugfea)
        #---concatenate embeddings at different layers (0 and l)
        cellpos = torch.cat((cellpos, cellfea), 1)
        drugpos = torch.cat((drugpos, drugfea), 1)
        #---inner product
        pos_adj = torch.matmul(cellpos, drugpos.t())
        pos_adj = self.act(pos_adj)
        return pos_z, neg_z, summary_pos, summary_neg, pos_adj.view(-1)

    def discriminate(self, z, summary, sigmoid=True):
        value = torch.matmul(z, torch.matmul(self.weight, summary))
        return torch.sigmoid(value) if sigmoid else value

    def loss(self, pos_z, neg_z, summary):
        pos_loss = -torch.log(
            self.discriminate(pos_z, summary, sigmoid=False) + EPS).mean()
        neg_loss = -torch.log(
            1 - self.discriminate(neg_z, summary, sigmoid=False) + EPS).mean()
        return pos_loss + neg_loss

    def __repr__(self):
        return '{}({})'.format(self.__class__.__name__, self.hidden_channels)

In [ ]:
model = GraphCDRUpdated(hidden_channels=256, encoder=Encoder(100, 256), summary=Summary(100, 256),
                 feat=NodeRepresentationUpdated(gcn_layer=atom_shape,output=100, use_GMP=False, use_gexpr=False,use_methylation=False),index=nb_celllines)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.1)

In [ ]:
import torch.nn as nn
myloss = nn.MSELoss()

In [ ]:
from scipy.stats import pearsonr, spearmanr

In [ ]:

def train():
    model.train()
    loss_temp=0
    for batch, (drug,cell) in enumerate(zip(drug_set,cellline_set)):
        optimizer.zero_grad()
        pos_z, neg_z, summary_pos, summary_neg, pos_adj=model(drug.x, drug.edge_index, drug.batch, cell[0],None, None, train_edge)
        dgi_pos = model.loss(pos_z, neg_z, summary_pos)
        dgi_neg = model.loss(neg_z, pos_z, summary_neg)
        pos_loss = myloss(pos_adj[train_mask],label_pos[train_mask])
        # loss=(1-0.3-0.3)*pos_loss + 0.3*dgi_pos + 0.3*dgi_neg
        loss = pos_loss
        loss.backward()
        optimizer.step()
        loss_temp += loss.item()
    print('train loss: ', str(round(loss_temp, 4)))

def test():
    model.eval()
    with torch.no_grad():
        yp = []
        for batch, (drug, cell) in enumerate(zip(drug_set, cellline_set)):
            _, _, _, _, pre_adj=model(drug.x, drug.edge_index, drug.batch,cell[0], None, None,train_edge)
            loss_temp = myloss(pre_adj[test_mask],label_pos[test_mask])
            yp.append(pre_adj[test_mask].detach().numpy())
        # yp=pre_adj[test_mask]#.detach().numpy()
        ytest=label_pos[test_mask]#.detach().numpy()
        # AUC, AUPR, F1, ACC =metrics_graph(ytest,yp)
        # print('test loss: ', str(round(loss_temp.item(), 4)))
        print(myloss(torch.Tensor(yp), ytest))
        # print('test auc: ' + str(round(AUC, 4)) + '  test aupr: ' + str(round(AUPR, 4)) +
        #       '  test f1: ' + str(round(F1, 4)) + '  test acc: ' + str(round(ACC, 4)))
    return myloss(torch.Tensor(yp), ytest).item()

#------main
final_loss = 0
for epoch in range(10):
    print('\nepoch: ' + str(epoch))
    train()
    f_loss = test()
    if (f_loss < final_loss):
        final_loss = f_loss



epoch: 0


/usr/local/lib/python3.9/dist-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


train loss:  0.0225


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([11068])) that is different to the input size (torch.Size([1, 11068])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.0248)

epoch: 1
train loss:  0.0223
tensor(0.0246)

epoch: 2
train loss:  0.0223
tensor(0.0244)

epoch: 3
train loss:  0.0223
tensor(0.0242)

epoch: 4
train loss:  0.0223
tensor(0.0240)

epoch: 5
train loss:  0.0223
tensor(0.0239)

epoch: 6
train loss:  0.0223
tensor(0.0238)

epoch: 7
train loss:  0.0223
tensor(0.0236)

epoch: 8
train loss:  0.0223
tensor(0.0235)

epoch: 9
train loss:  0.0224
tensor(0.0234)


In [ ]:
test()

tensor(0.0234)


0.0234376210719347

In [78]:
model.eval()
with torch.no_grad():
    yp = []
    for batch, (drug, cell) in enumerate(zip(drug_set, cellline_set)):
        _, _, _, _, pre_adj=model(drug.x, drug.edge_index, drug.batch,cell[0], None, None,train_edge)
        loss_temp = myloss(pre_adj[test_mask],label_pos[test_mask])
        yp.append(pre_adj[test_mask].detach().numpy())
    # yp=pre_adj[test_mask]#.detach().numpy()
    ytest=label_pos[test_mask]#.detach().numpy()
    # AUC, AUPR, F1, ACC =metrics_graph(ytest,yp)
    # print('test loss: ', str(round(loss_temp.item(), 4)))
    print(spearmanr(yp[0], ytest.detach().numpy()))
    print(pearsonr(yp[0], ytest.detach().numpy()))

/usr/local/lib/python3.9/dist-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


SpearmanrResult(correlation=0.07749184078780333, pvalue=3.245064044507172e-16)
(0.05492252263593628, 7.390051835134987e-09)
